In [ ]:
# Standard library imports.
import itertools
import json
import pickle

# Related third party imports.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (12,6)


# Local application/library specific imports.

In [ ]:
h = pickle.load(open('../results/25_italy/2022-11-11-00-08-10.p', "rb"))

In [ ]:
h

In [ ]:
pd.DataFrame(h)